# < Object Detection - YOLOv5>
- [yolov5 github](https://github.com/ultralytics/yolov5)

               ![YOLOv5_types](./data/jupyter_figure/yolov5_types.png)

# 0. Check GPU

In [ ]:
import torch #import pytorch packages
torch.cuda.is_available() # Check GPU is available
torch.cuda.get_device_name(0) # Check GPU Device Name

# 1. Image Inference : Use detect.py

- YOLOv5에서 기본적으로 제공해주는 detect.py를 사용한 객체 탐지
- detect.py는 coco 데이터셋으로 미리 학습된 yolov5s weights를 기본으로 사용함
- weights를 변경하고 싶을 경우(ex. 내가 학습시킨 모델) detect.py 실행 시 argument로 --weight 모델경로/모델파일이름.pt 를 넘겨주면 됨
- 객체 검출 결과는 yolov5/runs/detect/ 에 저장됨

### ※ COCO dataset & Annotation

- COCO 데이터셋은 대규모 Object Detection / Segmentation을 위한 데이터세트
- Object Detection 및 Segmentation 모델 학습 및 성능평가에 주로 사용되는 데이터셋
- 2014/2015/2017년 데이터셋이 존재하며 매년 이미지와 클래스 수가 증가하였음. 우리가 실습에 사용할 데이터셋은 2017 Validation Set
- Train Dataset : http://images.cocodataset.org/zips/train2017.zip
- Validation Dataset : http://images.cocodataset.org/zips/val2017.zip
- Test Dataset : http://images.cocodataset.org/zips/test2017.zip
- Annotation File : http://images.cocodataset.org/annotations/annotations_trainval2017.zip

### ※ YOLO format COCO dataset Annotation

- 일반적인 Object Detection의 데이터 어노테이션 파일은 json 형태의 파일 포멧을 사용함
- 하지만 YOLO 는 파일명.txt 형태의 텍스트 파일에 객체 클래스와 좌표를 적는 식의 독자적인 포멧을 사용함
- 따라서 YOLO format으로 변환된 coco dataset 어노테이션 파일이 필요하며, 이는 아래 링크에서 다운받을 수 있음
- Train / Validation Annotation File: https://ultralytics.com/assets/coco2017labels.zip
- Validation Image Dataset & Annotation File : https://ultralytics.com/assets/coco2017val.zip

### ※ detect.py Argument 목록 예시

``` shell
python detect.py --source 0  # webcam  
　　　　　　　　　　　　　img.jpg  # image  
　　　　　　　　　　　　　vid.mp4  # video  
　　　　　　　　　　　　　path/  # directory  
　　　　　　　　　　　　　path/*.jpg  # glob  
　　　　　　　　　　　　　'https://youtu.be/Zgi9g1ksQHc'  # YouTube  
　　　　　　　　　　　　　'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```
             
- 더 자세한 Argument는 detect.py를 열어보면 확인할 수 있음

In [ ]:
!cd yolov5 && python detect.py --source ../data/coco_val/images/val2017/000000404922.jpg

# 2. Video Inference : Use detect.py

In [ ]:
!cd yolov5 && python detect.py --source ../data/video_data/input_video.mp4

# 3. Inference Image using pytorch load model function

- YOLOv5 모델을 파이토치에서 직접 불러와 사용하는 방식.
- 입력과 출력에 대한 작업을 자유롭게 조정할 수 있다는 장점이 있다.
- 또한 객체 검출 뿐만 아니라 다른 부가적인 기능들을 구현해야 할때 사용이 용이하다. 

## 1) Load COCO Dataset

### (1) Import Packages

In [ ]:
import torch
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd

### (2) Load Image from COCO Dataset

In [ ]:
data_path = './data/coco_val/images/val2017/'
data_list = os.listdir(data_path)
data_name = '000000404922.jpg'

org_img = cv2.imread(data_path + data_name,cv2.IMREAD_COLOR)
org_img = cv2.cvtColor(org_img,cv2.COLOR_BGR2RGB)

result_img = cv2.imread(data_path + data_name,cv2.IMREAD_COLOR)
result_img = cv2.cvtColor(result_img,cv2.COLOR_BGR2RGB)

gt_img = cv2.imread(data_path + data_name,cv2.IMREAD_COLOR)
gt_img = cv2.cvtColor(gt_img,cv2.COLOR_BGR2RGB)

### (3) Plot loaded Images

In [ ]:
plt.imshow(org_img)

## 2) Object Detect by YOLO V5

### (1) Load Model

               ![YOLO v5](./data/jupyter_figure/YOLOv5_Architecture.png)

In [ ]:
model = torch.hub.load('ultralytics/yolov5','yolov5s') # or yolov5m, yolov5l, yolov5x, custom

### (2)  Localization & Predict Classes

In [ ]:
# Predict image 0
results = model(result_img)
df = results.pandas().xyxy[0]
df

In [ ]:
obj_list = []
for i in range(len(df)) :
    obj_xmin = int(df['xmin'][i])
    obj_ymin = int(df['ymin'][i])
    obj_xmax = int(df['xmax'][i])
    obj_ymax = int(df['ymax'][i])
    obj_confi = df['confidence'][i]
    obj_class = df['class'][i]
    obj_name = df['name'][i]
    obj_dict = {
                'class' : obj_name, 
                'confidence' : obj_confi, 
                'bbox' : [obj_xmin,obj_ymin,obj_xmax,obj_ymax]
    }
    obj_list.append(obj_dict)

In [ ]:
cnt = 0
for obj_dict in obj_list :
    print('[%d Object] \n - class name : %s \n - xmin : %d \n - xmax : %d \n - ymin : %d \n - ymax : %d \n - confidence : %f\n'%(cnt, obj_dict['class'], obj_dict['bbox'][0], obj_dict['bbox'][1], obj_dict['bbox'][2], obj_dict['bbox'][3], obj_dict['confidence']))
    cnt = cnt + 1

### (3) Draw Rectangle

In [ ]:
pred_color = {'person' : (255,0,0), #Red
              'tennis racket' : (0,0,255), #Blue
              'car' : (0,255,0) #Green
             }

In [ ]:
for obj_dict in obj_list : 
    #TO DO - thickness = 3
    
    
    cnt = cnt + 1

In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(result_img)

### (4) Write Class name & Confidence

In [ ]:
for obj_dict in obj_list :
    class_name = obj_dict['class']
    confidence = "%.2f"%(obj_dict['confidence'])
    text_x = obj_dict['bbox'][0] + 10 #xmin + 10
    text_y = obj_dict['bbox'][1] + 20 #ymin + 10
    font = cv2.FONT_HERSHEY_SIMPLEX
    #TO DO - fontscale = 0.7, thickness = 2
    
    
    
plt.figure(figsize = (15,8))
plt.imshow(result_img)

### (5) Compare result with Ground Truth

- Load COCO Dataset Annotation File

In [ ]:
import json
with open("./data/coco_val/annotations/instances_val2017.json","r") as json_file :
    annotation = json.load(json_file)

In [ ]:
data_id = data_name[6:].replace(".jpg","")
data_id

- Load Groud Truth Information from Annotation File

In [ ]:
anno_list = []
obj_dict = {}
print()
for anno_data in annotation['annotations'] :
    if anno_data['image_id'] == int(data_id) :
        obj_dict['image_id'] = anno_data['image_id']
        obj_dict['bbox'] = anno_data['bbox']
        obj_dict['category_id'] = anno_data['category_id']
        for categories in annotation['categories'] :
            if obj_dict['category_id'] == categories['id'] : 
                obj_dict['class'] = categories['name']
        anno_list.append(obj_dict)
        obj_dict = {}

In [ ]:
anno_list

- Change Coordinate (x,y,w,h) =====> (xmin,ymin,xmax,ymax)

                         ![Coordinate Examples](./data/jupyter_figure/coordinate_example.PNG)

In [ ]:
for anno_dict in anno_list :
    x = anno_dict['bbox'][0]
    y = anno_dict['bbox'][1]
    w = anno_dict['bbox'][2]
    h = anno_dict['bbox'][3]
    xmin = x
    ymin = y
    xmax = x + w
    ymax = y + h
    change_coord = list(map(int,[xmin,ymin,xmax,ymax]))
    anno_dict['bbox'] = change_coord

In [ ]:
anno_list

In [ ]:
gt_color = {'person' : (255,105,180), # hotpink
            'tennis racket' : (135,206,235), # skyblue
            #'bench' : (192,192,192), #Silver
             }

In [ ]:
for anno_dict in anno_list : 
    cv2.rectangle(gt_img, (anno_dict['bbox'][2],anno_dict['bbox'][3]), (anno_dict['bbox'][0],anno_dict['bbox'][1]), color=gt_color[anno_dict['class']],thickness=3)
    cnt = cnt + 1

In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(gt_img)

In [ ]:
for anno_dict in anno_list :
    class_name = anno_dict['class']
    text_x = anno_dict['bbox'][0] + 10
    text_y = anno_dict['bbox'][1] + 20
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(gt_img,class_name,(text_x,text_y),font,0.7,gt_color[class_name],2)
    
plt.figure(figsize = (10,6))
plt.imshow(gt_img)

- Compare Groud Truth & Prediction Image

In [ ]:
img_list = [org_img,result_img,gt_img]
fig = plt.figure(figsize=(25,8))
rows = 1
cols = 3
xlabels = ['Original Image', 'Result Image', 'Ground Truth Image']
for i in range(len(img_list)):
    ax = fig.add_subplot(rows, cols, i+1)
    ax.imshow(img_list[i])
    ax.set_xlabel(xlabels[i],fontsize = 15)
    ax.set_xticks([]), ax.set_yticks([])
    i += 1

# 4. Evaluate Object Detection

In [ ]:
!python ./yolov5/val.py --weights yolov5s.pt --data coco.yaml --img 640 --iou 0.65 --half

## <Appendix - Calculate IoU & Recall & Precision & mAP>

### (1) IOU(Intersection over Union) & TP(True Positive)/FP(False Positive)

![IoU Examples](./data/jupyter_figure/IoU.png)              ![IoU Examples](./data/jupyter_figure/IoU_example.png)!

$$IoU = {area(B_{gt} \cap B_{pred}) \over area(B_{gt} \cup B_{pred})}$$

- Confusion Matrix by IoU

![IoU Examples](./data/jupyter_figure/Confusion_Matrix.png)

- TP(True Positive : 실제 양성 / 예측 양성 - 검출되어야 할 영역이 잘 검출되었음) : 올바른 탐지 [IoU >= Threshold]
- FP(False Positive : 실제 음성 / 예측 양성 - 객체가 없는 영역에 검출이 되었음) : 오탐지 [IoU <= Threshold] (IoU가 임계값 보다 낮다는 것은 객체가 없는 곳에 예측 B-box를 그렸단 것이므로 FP가 된다)
- FN(False Negative : 실제 양성 / 예측 음성 - 객체가 있는 영역에 검출이 되지 않음) : 미탐지 (Ground Truth 에는 라벨링 된 Box가 존재하나, Detector가 탐지하지 못한 경우)
- TN(True Negative : 실제 음성 / 예측 음성 - 객체가 없는 영역에 검출이 되지 않음) : Object Detection에서 해당 지표는 사용하지 않음. 객체가 없는 영역이란 고려하지 않기 때문

In [ ]:
#Check Image Size
org_img.shape

In [ ]:
#Get Width / Height
img_h = org_img.shape[0]
img_w = org_img.shape[1]

#Make white Image
iou_image = np.full((img_h,img_w,3),255,np.uint8)

plt.imshow(iou_image)

In [ ]:
#Draw Predicted result B-box
for obj_dict in obj_list : 
    cv2.rectangle(iou_image, (obj_dict['bbox'][2],obj_dict['bbox'][3]), (obj_dict['bbox'][0],obj_dict['bbox'][1]), color=pred_color[obj_dict['class']],thickness=2)
    cnt = cnt + 1

#Draw Ground Truth B-box 
for anno_dict in anno_list : 
    cv2.rectangle(iou_image, (anno_dict['bbox'][2],anno_dict['bbox'][3]), (anno_dict['bbox'][0],anno_dict['bbox'][1]), color=gt_color[anno_dict['class']],thickness=2)
    cnt = cnt + 1
    
plt.figure(figsize=(15,6))
plt.imshow(iou_image)

In [ ]:
#Get Prediction classes
pred_classes = []
for obj_dict in obj_list :
    if  obj_dict['class'] not in pred_classes :
        pred_classes.append(obj_dict['class'])
pred_classes

In [ ]:
#Split obj_List by pred_classes
class_objdict = {}
for i in range(len(pred_classes)) :
    class_List = []
    for obj_dict in obj_list :
        if pred_classes[i] == obj_dict['class'] :
            class_List.append(obj_dict)
    class_objdict[class_List[0]['class']] = class_List
class_objdict

In [ ]:
# Sorted by Confidence for all class lists
for list_class in class_objdict.values() : 
    list_class.sort(key=lambda x:x['confidence'],reverse = True)

- Calculate IoU, TP/FP for Person class

                  ![Intersection_coordinates.PNG](./data/jupyter_figure/Intersection_coordinates.PNG)   

In [ ]:
# Make Dataframe
df_dict = {}
for i in range(len(pred_classes)) : 
    df = pd.DataFrame(columns=['Detected class','confidence','IoU','TP or FP'])
    df_dict[pred_classes[i]] = df
df_dict

In [ ]:
#Find Best IoU with Ground Truth for each predicted B-box
for key,class_objlist in class_objdict.items() : 
    for class_dict in class_objlist :
        best_iou = 0
        best_Anno = None
        for anno_dict in anno_list :
            #x1,y1 - predicted B-box
            min_x1 = class_dict['bbox'][0]
            min_y1 = class_dict['bbox'][1]
            max_x1 = class_dict['bbox'][2]
            max_y1 = class_dict['bbox'][3]

            #x2,y2 - Ground Truth B-box
            min_x2 = anno_dict['bbox'][0]
            min_y2 = anno_dict['bbox'][1]
            max_x2 = anno_dict['bbox'][2]
            max_y2 = anno_dict['bbox'][3]

            #TO DO - set intersection coordinate
            interMin_x = max(min_x1,min_x2)
            interMin_y = max(min_y1,min_y2)
            interMax_x = min(max_x1,max_x2)
            interMax_y = min(max_y1,max_y2)

            #Calculate intersection area
            inter_w = (interMax_x - interMin_x)
            inter_h = (interMax_y - interMin_y)
            if inter_w < 0 :
                inter_w = 0
                
            if inter_h < 0 :
                inter_h = 0
                
            intersection = inter_w * inter_h
            #Calculate union area
            union = ((max_x1 - min_x1) * (max_y1 - min_y1)) + ((max_x2 - min_x2) * (max_y2 - min_y2)) - intersection
            
            #Calculate IoU
            iou = intersection / union
            if best_iou < iou : 
                best_iouAnno = anno_dict #for check predict class == gt class
                best_iou = iou
                #print(best_iou)
        #print(best_iou)
        if best_iou >= 0.5 and class_dict['class'] == best_iouAnno['class'] : #check iou >= threshold & predict class == gt class
            result = 'TP'
        else :
            result = 'FP'

        row = [class_dict['class'],class_dict['confidence'],best_iou,result]
        df_dict[key].loc[len(df_dict[key])] = row

In [ ]:
for class_name in pred_classes :
    df_dict[class_name] = df_dict[class_name].sort_values('confidence',ascending=False)
    df_dict[class_name] = df_dict[class_name].reset_index(drop=True) #행번호 정렬

In [ ]:
from tabulate import tabulate
for key,values in df_dict.items() :
    print('[' + key +']')
    print(tabulate(values,headers='keys',tablefmt='psql',showindex=True))
    print()

- Calculate IoU, TP/FP for Frisbee class

### (2) Accumulate TP and FP

In [ ]:
#Add Accumulate TP / Accumulate FP Column
for value in df_dict.values() :
    accTP_col = [pd.NA for i in range(len(value))]
    accFP_col = [pd.NA for i in range(len(value))]
    value['Accumulate TP'] = accTP_col
    value['Accumulate FP'] = accFP_col

In [ ]:
from tabulate import tabulate
for key,values in df_dict.items() :
    print('[' + key +']')
    print(tabulate(values,headers='keys',tablefmt='psql',showindex=True))
    print()

In [ ]:
#Calculate Accumulate TP or FP
pd.set_option('mode.chained_assignment',  None) #Off Warnings

for df in df_dict.values() : 
    cnt = 0
    tporfp = df['TP or FP']
    num_TP = 0
    num_FP = 0
    for tf in tporfp : 
        if tf == 'TP' :
            if num_TP == 0 and num_FP == 0 :
                num_TP = 1
                num_FP = 0
            else :
                num_TP = df['Accumulate TP'][cnt-1] + 1
                num_FP = df['Accumulate FP'][cnt-1]
        else :
            if num_TP == 0 and num_FP == 0 :
                num_TP = 0
                num_FP = 1
            else :
                num_TP = df['Accumulate TP'][cnt-1]
                num_FP = df['Accumulate FP'][cnt-1] + 1
        df['Accumulate TP'][cnt] = num_TP
        df['Accumulate FP'][cnt] = num_FP
        cnt = cnt + 1        

In [ ]:
for key,values in df_dict.items() :
    print('[' + key +']')
    print(tabulate(values,headers='keys',tablefmt='psql',showindex=True))
    print()

### (3) Precision

- Precision(정밀도) 는 모델 예측 값이 양성(Positive)인 대상 중 예측과 실제 값이 양성으로 일치하는 비율
- 모델이 얼마나 정확한지를 측정하는 척도
- 예) 모델이 객체로 검출한 수 : 10, 그 중 올바르게 검출한 갯수 : 5 => Precision = 0.5

$$Precision = { TP \over TP + FP} = {TP \over All\,Detections}$$

In [ ]:
precision_Dict = {}
for key, class_dict in df_dict.items() : 
    precision_List = []
    for (acc_TP, acc_FP) in zip(class_dict['Accumulate TP'],class_dict['Accumulate FP']) :
        precision = acc_TP / (acc_TP + acc_FP)
        precision_List.append(precision)
    precision_Dict[key] = precision_List
precision_Dict

### (4) Recall

- Recall(재현율)은 실제 값이 양성(Positive)인 대상 중 예측과 실제값이 양성으로 일치하는 비율
- 모델이 얼마나 양성(Positive) 값들을 잘 찾는지 측정하는 척도
- 예) 검출해야하는 객체 수 : 10개, 모델이 객체로 올바르게 검출한 수 : 7 => Recall = 0.7

$$Recall = {TP \over TP + FN} = {TP \over All\,Ground\,Truths}$$

In [ ]:
import collections
#make annotation class num list
anno_classList = []
for anno_dict in anno_list :
    anno_classList.append(anno_dict['class'])

#Count number of annotation classes
gt_number = collections.Counter(anno_classList)

print("All annotation Class list : " + str(anno_classList))
print("Prediction Class Type List : " + str(pred_classes))
print(gt_number)

In [ ]:
#Add Wrong predict classes in to anno_classList 
gt_number = dict(gt_number) # Change to Dictionary

#Find wrong predict classes through compare with anno_classList
wrongpred_classList = []
for pred_class in pred_classes :
    if pred_class not in gt_number.keys() :
        gt_number[pred_class] = 0
print(gt_number)

In [ ]:
recall_Dict = {}
for key, class_dict in df_dict.items() :
    recall_List = []
    for (acc_TP, acc_FP) in zip(class_dict['Accumulate TP'],class_dict['Accumulate FP']) :
        if gt_number[key]!=0 : 
            recall = acc_TP / gt_number[key] #All Ground Truths(per Classes)
            recall_List.append(recall)
        else : #Prevent Division by Zero
            recall = 0
            recall_List.append(recall)
    recall_Dict[key] = recall_List
recall_Dict

### (5) PR Curve

- Precision 과 Recall 은 반비례의 관계를 가짐
- 정확도와 검출율의 성능 변화를 확인하기 위해 X축을 Recall으로, Y 축을 Precision 으로 한 PR 곡선을 사용함

In [ ]:
fig = plt.figure(figsize = (20,10))
xlabels = ['Person PR-Curve', 'Tennis Racket PR-Curve', 'Car PR-Curve']
rows = 2
cols = 2
cnt = 0
for precision_List, recall_List in zip(precision_Dict.values(),recall_Dict.values()) :
    ax = fig.add_subplot(rows,cols, cnt+1)
    ax.scatter(recall_List,precision_List,color = 'red')
    ax.plot(recall_List,precision_List,label = 'PR-Curve')
    ax.set_xlabel(xlabels[cnt], fontsize = 15)
    ax.set_ylim(-0.1,1.1)
    ax.grid()
    ax.legend()
    cnt = cnt + 1

### (6) AP

- PR 곡선을 단조적으로 감소하도록 만든 그래프의 면적

In [ ]:
for key, precision_List in precision_Dict.items() : 
    tmp = -1 
    for i in range(len(precision_List)-1) :
        if precision_List[i] < precision_List[i+1] :
            precision_List[i] = precision_List[i+1]
    precision_Dict[key] = precision_List
precision_Dict

In [ ]:
fig = plt.figure(figsize = (20,20))
rows = 3
cols = 2
cnt = 0
for (recall_key,recall_List), (precision_key,precision_List) in zip(recall_Dict.items(), precision_Dict.items()) : 
    #Plot Monotonic Decrease PR-Curve
    ax1 = fig.add_subplot(rows,cols,cnt+1)
    ax1.set_title("Monotonic Decrease PR-Curve [" + recall_key + "]" , fontsize = 20)
    ax1.plot(recall_List,precision_List,label = 'Monotonic Decrease PR-Curve')
    ax1.scatter(recall_List,precision_List,color = 'red')
    ax1.set_ylim(-0.1,1.1)
    ax1.set_xlabel("Recall",fontsize=20)
    ax1.set_ylabel("Precision",fontsize=20)
    ax1.legend()
    ax1.grid()
    cnt = cnt + 1
    
    #Plot Area of Graph
    ax2 = fig.add_subplot(rows,cols,cnt+1)
    ax2.set_title("Monotonic Decrease PR-Curve Area [" + recall_key + "]" , fontsize = 20)
    ax2.plot(recall_List,precision_List,label = 'Monotonic Decrease PR-Curve')
    ax2.scatter(recall_List,precision_List,color = 'red')
    ax2.fill_between(recall_List,precision_List)
    ax2.set_xlabel("Recall",fontsize=20)
    ax2.set_ylabel("Precision",fontsize=20)
    ax2.legend()
    cnt = cnt + 1
plt.show()

In [ ]:
#Calculate AP and make AP DICT
ap_dict = {}
for (precision_key, precision), (recall_key, recall) in zip(precision_Dict.items(),recall_Dict.items()) :
    precision = torch.Tensor(precision)
    recall = torch.Tensor(recall)
    ap = torch.trapz(precision,recall) #Calculate AP by Integral f(x,y) / trapz(y,x) #
    ap_dict[precision_key] = float(ap)

#Print classes's AP
for ap_class, ap in ap_dict.items() :
    print("AP of " + ap_class + " : " + str(ap))

### (7) mAP

- 각 클래스 별 AP의 평균
- mAP가 높을 수록 검출 모델의 성능이 좋은 것으로 판단

In [ ]:
sum_ap = sum(ap_dict.values())
mAP = sum_ap/len(pred_classes)
print("mAP of YOLO v5s for this image : %.2f %%" %(mAP*100))